# 네이버 지도 리뷰 크롤링

In [24]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from user_agent import generate_user_agent
from tqdm.notebook import tqdm
from bs4 import BeautifulSoup
from urllib.request import Request
import time
import pandas as pd
import re
import random

* 서울시 반려동물 동반 가능 시설 데이터 Load

In [3]:
df = pd.read_csv("../data/preprocessed_per_culture_site.csv", index_col=0)
df.head()

,시설명,카테고리,주소,위도,경도
0,100세약국,동물약국,서울특별시 영등포구 당산동6가 217-4,37.533260,126.903741
1,100코달리 와인바,카페,서울특별시 중구 신당동 292-152,37.563305,127.016417
2,1층메디컬약국,동물약국,서울특별시 금천구 시흥동 889-8,37.453043,126.901712
3,1층이화약국,동물약국,서울특별시 성동구 하왕십리동 1070,37.566590,127.024082
4,21세기동물병원,동물병원,서울특별시 용산구 보광동 259-1,37.528316,126.999196


* 동물약국은 제외한다

In [4]:
df = df[df["카테고리"]!="동물약국"]
df.reset_index(drop=True, inplace=True)
df.head()

,시설명,카테고리,주소,위도,경도
0,100코달리 와인바,카페,서울특별시 중구 신당동 292-152,37.563305,127.016417
1,21세기동물병원,동물병원,서울특별시 용산구 보광동 259-1,37.528316,126.999196
2,24시 강서 젠틀리동물의료센터,동물병원,서울특별시 강서구 마곡동 747,37.559040,126.820063
3,24시 굿케어동물의료센터,동물병원,서울특별시 관악구 봉천동 1673-21,37.480000,126.956867
4,24시 글로리 동물병원,동물병원,서울특별시 금천구 독산동 145-13,37.476744,126.898256


* 추후 리뷰 크롤링을 위해 고유 id값을 담을 컬럼 추가

In [5]:
df["인프라id"] = ""
df.head()

,시설명,카테고리,주소,위도,경도,인프라id
0,100코달리 와인바,카페,서울특별시 중구 신당동 292-152,37.563305,127.016417,
1,21세기동물병원,동물병원,서울특별시 용산구 보광동 259-1,37.528316,126.999196,
2,24시 강서 젠틀리동물의료센터,동물병원,서울특별시 강서구 마곡동 747,37.559040,126.820063,
3,24시 굿케어동물의료센터,동물병원,서울특별시 관악구 봉천동 1673-21,37.480000,126.956867,
4,24시 글로리 동물병원,동물병원,서울특별시 금천구 독산동 145-13,37.476744,126.898256,


* 데이터프레임에 담긴 시설을 네이버지도에서 검색하여 고유 id 값을 가져와서 인프라id 컬럼에 추가

In [18]:
driver = webdriver.Chrome(service=Service("../driver/chromedriver"))
id_src = ""

for _, row in df.iterrows():
    url = "https://map.naver.com/p/search/" + str(row["시설명"]).replace("24시", "") + " " + row["주소"] + "?c=15.00,0,0,0,dh"

    driver.get(url)

    try:
        id_src = WebDriverWait(driver, 2).until(EC.presence_of_element_located((By.ID, "entryIframe"))).get_attribute("src")
    except:
        continue

    id = re.findall("\/(\d+)?", id_src)[3]
    df.loc[df["시설명"]==row["시설명"], "인프라id"] = id

driver.quit()

https://pcmap.place.naver.com/place/36399514?from=map&fromPanelNum=1&additionalHeight=76&timestamp=202412271735&locale=ko
https://pcmap.place.naver.com/place/87231883?from=map&fromPanelNum=1&additionalHeight=76&timestamp=202412271736&locale=ko


NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=131.0.6778.204)
Stacktrace:
#0 0x64352295e8fa <unknown>
#1 0x64352246fd20 <unknown>
#2 0x6435224459cf <unknown>
#3 0x6435224eaf2d <unknown>
#4 0x643522500f1c <unknown>
#5 0x6435224e2893 <unknown>
#6 0x6435224b130d <unknown>
#7 0x6435224b232e <unknown>
#8 0x64352292b00b <unknown>
#9 0x64352292ef97 <unknown>
#10 0x64352291771c <unknown>
#11 0x64352292fb17 <unknown>
#12 0x6435228fc6cf <unknown>
#13 0x64352294d6b8 <unknown>
#14 0x64352294d880 <unknown>
#15 0x64352295d776 <unknown>
#16 0x7b12f8c94ac3 <unknown>


* 인프라id가 잘 담겼는지 확인

In [6]:
df

,시설명,카테고리,주소,위도,경도,인프라id
0,100코달리 와인바,카페,서울특별시 중구 신당동 292-152,37.563305,127.016417,
1,21세기동물병원,동물병원,서울특별시 용산구 보광동 259-1,37.528316,126.999196,
2,24시 강서 젠틀리동물의료센터,동물병원,서울특별시 강서구 마곡동 747,37.559040,126.820063,
3,24시 굿케어동물의료센터,동물병원,서울특별시 관악구 봉천동 1673-21,37.480000,126.956867,
4,24시 글로리 동물병원,동물병원,서울특별시 금천구 독산동 145-13,37.476744,126.898256,
...,...,...,...,...,...,...
2115,VIP 동물의료센터,동물병원,서울특별시 성북구 동소문동6가 3-1,37.592296,127.013413,
2116,VIP동물의료센터,동물병원,서울특별시 동대문구 장안동 335-3,37.572010,127.073563,
2117,VIP동물의료센터 서초동작점,동물병원,서울특별시 서초구 방배동 442-3,37.479673,126.982192,
2118,VIP동물의료센터 청담점,동물병원,서울특별시 강남구 청담동 46,37.519471,127.048985,


* 해당 데이터 저장

In [3]:
df.to_csv("../data/naver_review_data.csv")

NameError: name 'df' is not defined

In [2]:
df = pd.read_csv("../data/naver_review_data.csv", index_col=0, dtype={'인프라id':object})
df.reset_index(drop=True, inplace=True)

df.head()

,시설명,카테고리,주소,위도,경도,인프라id
0,100코달리 와인바,카페,서울특별시 중구 신당동 292-152,37.563305,127.016417,NaN
1,21세기동물병원,동물병원,서울특별시 용산구 보광동 259-1,37.528316,126.999196,NaN
2,24시 강서 젠틀리동물의료센터,동물병원,서울특별시 강서구 마곡동 747,37.559040,126.820063,36399514
3,24시 굿케어동물의료센터,동물병원,서울특별시 관악구 봉천동 1673-21,37.480000,126.956867,1347656288
4,24시 글로리 동물병원,동물병원,서울특별시 금천구 독산동 145-13,37.476744,126.898256,87231883


* 인프라id에 Nan이 들어간 행은 네이버 지도에서 검색되지 않는 시설이다(필터링)

In [3]:
df = df[df["인프라id"].isna() == False]
df.reset_index(drop=True, inplace=True)
df

,시설명,카테고리,주소,위도,경도,인프라id
0,24시 강서 젠틀리동물의료센터,동물병원,서울특별시 강서구 마곡동 747,37.559040,126.820063,36399514
1,24시 굿케어동물의료센터,동물병원,서울특별시 관악구 봉천동 1673-21,37.480000,126.956867,1347656288
2,24시 글로리 동물병원,동물병원,서울특별시 금천구 독산동 145-13,37.476744,126.898256,87231883
3,24시 산들산들 동물병원,동물병원,서울특별시 마포구 아현동 424-14,37.550975,126.956497,37371383
4,24시 서울 탑 동물병원,동물병원,서울특별시 양천구 신정동 1182-11,37.517743,126.853643,715618768
...,...,...,...,...,...,...
768,staywell cafe,카페,서울특별시 마포구 공덕동 476,37.544748,126.948258,1038521844
769,uphill namsan,식당,서울특별시 용산구 이태원동 5-27,37.540077,126.994298,1125647880
770,VIP 동물의료센터,동물병원,서울특별시 성북구 동소문동6가 3-1,37.592296,127.013413,13094638
771,VIP동물의료센터 청담점,동물병원,서울특별시 강남구 청담동 46,37.519471,127.048985,1192144496


* 셀레니움에서 리뷰 더보기 버튼을 눌러주는 함수

In [30]:
def morebtn_click(proc_driver):
    action = ActionChains(proc_driver)

    while True:
        try:
            more_btn = WebDriverWait(proc_driver, 5).until(EC.element_to_be_clickable((By.CLASS_NAME, "TeItc")))
            proc_driver.execute_script("arguments[0].click();", more_btn)
        except:
            print("click done....")
            return True
        
    return False
    

* 이모지를 걸러내기 위한 함수

In [5]:
def remove_emojis(text):
    pattern = r'[^0-9a-zA-Z가-힣!@#\$%\^&\*\(\)\-_=+\{\}\[\]:;"\'<>,\.\/?`~ ]+'
    filter_text = re.sub(pattern, '', text)

    return filter_text

* 리뷰를 가져오는 함수

In [25]:
def get_review(html):
    review_list = []
    """
    li_list = driver.find_elements(By.CLASS_NAME, "pui__X35jYm.place_apply_pui.EjjAW")

    for comment in li_list:
        try:
            comment_tmp = comment.find_element(By.CLASS_NAME, "pui__vn15t2").text
        except Exception as e:
            print("에러 발생")
            continue
        if not comment_tmp == "":
            comment_tmp = comment_tmp.replace("\n", " ")
            comment_tmp = comment_tmp.replace("더보기", "")
            review_list.append(remove_emojis(comment_tmp))
    """

    req = Request(url, headers={"User-Agent":generate_user_agent()})
    soup = BeautifulSoup(html, "html.parser")

    review_li = soup.find("div", "place_section k1QQ5")
    for row in review_li.find_all("li", "pui__X35jYm place_apply_pui EjjAW"):
        review_list.append(row.find("div", "pui__vn15t2").get_text())
            
    print("리뷰수 : " + str(len(review_list)))
    return review_list

In [35]:
review_dict = {}

* 데이터프레임에 담긴 시설 정보를 토대로 네이버 플레이스에서 리뷰를 크롤링해서 딕셔너리에 저장

In [34]:
options = Options()
options.add_argument('--disable-images')
options.add_experimental_option("prefs", {'profile.managed_default_content_settings.images': 2})
options.add_argument('--blink-settings=imagesEnabled=false')
#options.add_argument(f"user-agent={generate_user_agent()}")
options.add_argument('--headless') 

driver = webdriver.Chrome(service=Service("../driver/chromedriver"), options=options)
driver.delete_all_cookies()

for _, row in tqdm(df.iloc[:1].iterrows(), total=len(df.iloc[:1].index)):
    url = "https://pcmap.place.naver.com/place/" + str(row["인프라id"]) + "/review/visitor"
    driver.get(url)

    if "Too Many Requests" in driver.page_source:
         print(driver.page_source)
         break

    print("[" + row["시설명"] + "] processing...")

    if morebtn_click(driver):
         review_list = get_review(driver.page_source)

    review_dict[row["시설명"]] = review_list

    time.sleep(random.uniform(0.1, 0.3))

driver.quit()

  0%|          | 0/1 [00:00<?, ?it/s]

[24시 강서 젠틀리동물의료센터] processing...
click done....
리뷰수 : 593


NameError: name 'review_dict' is not defined

In [11]:
review_dict.keys()

dict_keys([])

* 딕셔너리에 저장된 시설에 대한 리뷰를 풀어서 데이터프레임으로 구성

In [51]:
data = []

for name, list in review_dict.items():
    for review in list:
        data.append({"시설명":name, "리뷰":review})

review_df = pd.DataFrame(data)
review_df

,시설명,리뷰
0,24시 강서 젠틀리동물의료센터,뼈다귀 맛집
1,24시 강서 젠틀리동물의료센터,굿
2,24시 강서 젠틀리동물의료센터,애기아파서 들렀는데 친절하게 잘 상담해주셨어요
3,24시 강서 젠틀리동물의료센터,야간에도 진료를 잘봐줘요
4,24시 강서 젠틀리동물의료센터,좋아요
...,...,...
1326,24시 글로리 동물병원,
1327,24시 글로리 동물병원,^^
1328,24시 글로리 동물병원,늦은 시간에 방문했는데도 너무 친절하게 진료 봐주셨어요 겁많은 저희 냥이도 순둥하게...
1329,24시 글로리 동물병원,봉구 심장사상충 맞고 왔어요


* 리뷰가 공백인 경우를 필터링(사진만 올렸거나 이모지만 작성하여 필터링된 경우)

In [53]:
review_df = review_df[review_df["리뷰"] != ""]
review_df.reset_index(drop=True, inplace=True)
review_df

,시설명,리뷰
0,24시 강서 젠틀리동물의료센터,뼈다귀 맛집
1,24시 강서 젠틀리동물의료센터,굿
2,24시 강서 젠틀리동물의료센터,애기아파서 들렀는데 친절하게 잘 상담해주셨어요
3,24시 강서 젠틀리동물의료센터,야간에도 진료를 잘봐줘요
4,24시 강서 젠틀리동물의료센터,좋아요
...,...,...
1311,24시 글로리 동물병원,믿고 가는 동물병원 :)
1312,24시 글로리 동물병원,^^
1313,24시 글로리 동물병원,늦은 시간에 방문했는데도 너무 친절하게 진료 봐주셨어요 겁많은 저희 냥이도 순둥하게...
1314,24시 글로리 동물병원,봉구 심장사상충 맞고 왔어요


In [23]:
url = "https://pcmap.place.naver.com/place/36399514/review/visitor"
driver = webdriver.Chrome(service=Service("../driver/chromedriver"))
driver.get(url)

In [25]:
while True:
    try:
        more_btn = driver.find_element(By.CLASS_NAME, "TeItc")
        data = driver.execute_script("arguments[0].click();", more_btn)
    except:
        break

In [19]:
driver.quit()

In [33]:
driver = webdriver.Chrome(service=Service("../driver/chromedriver"))

url = "https://pcmap.place.naver.com/place/1347656288/review/visitor"
driver.get(url)
morebtn_click(driver)
req = Request(url, headers={"User-Agent":generate_user_agent()})
soup = BeautifulSoup(driver.page_source, "html.parser")

review_li = soup.find("div", "place_section k1QQ5")
for row in review_li.find_all("li", "pui__X35jYm place_apply_pui EjjAW"):
    print(row.find("div", "pui__vn15t2").get_text())

driver.quit()

click done....
수의사 선생님들 너~~~무 친절하게 잘 봐주세요👍❤️과잉진료 없습니다!
애기 심장사상충 하러 자주 방문 하는데 그냥 사료만 사러갈때도 데스크 분들께서 엄청 친절하게 잘 대해주시는것 같아요 잘 몰랐던거 여쭤봤는데 친절히 상세하게 알려주셔서 너무 감사했습니다 더 자주 방문할거같아요 ㅎㅎ
여긴 정말 카운터에있는 분들이 제일 문제인 병원같아요. 말투도 항상 거슬리게 말씀하시고 뭐 물어보면 알지도 못하면서 원장님한테 여쭤보지도 않고 말씀해주시구요.어쩔수없는 상황이라 계속 병원은 다니고 있지만 3년동안 변함이없네요. 본인들 반려견 반려묘였어도 말을 그렇게 할수있는지 궁금하네요. 원장님들이 좋아서 다니고는 있지만 늘 갈때마다 기분상해요. 후기 보시면서 느끼시는게 없나요? 저처럼 생각하는분들이 한두분이 아닌거같은데..아니면 후기를 안보시나요? 제발 후기좀 보시고 변화가 있었으면 좋겠네요. 한번만 더 기분상하면 안다닐려고 마음먹고 적습니다.더보기
친절
친절하고 자세한 상담해주셔서 계속 방문하게 되네요!
굿
선생님들 친절해요
대기도 많이없고 조아요
항상 친절 대기적고 믿응직
근처에서 젤 믿음직스러워요
수술잘시켜주셨습니다
밤에 이용 가능해 좋아요
친절해요
전문가
24시라 응급할때 좋아요
.
사료사러왔어요
너무 안 좋은 상태에서 찾아갔었는데 이지혁선생님 덕분에 부엉이가 며칠동안 다시 기력도 회복하고 그루밍도하고 사냥도 하면서 잘 지냈어요 이번에도 잘 버텨주면 좋겠는데,, 위급할때마다 필요한 치료해주시고 이해하기 쉽게 설명도 잘해주셔서 감사해요  
친절하게 상담해주시고진료 해주시고간호사님도 친절해요
강아지가 아파서 가까운 병원이라 자주 방문하고 있는데 선생님들 모두 친절하시고 인포에서도 늘 반갑게 맞아주시고 애기도 너무 예뻐해주셔서 좋은 기억으로 다시 방문 하게 되네요~ 항상 감사합니다. 🥰
건강검진차 내원하였는데 비용부담은 조금 있었지만  꼼꼼하고 자세한 설명과 깨끗한 환경, 친절한 서비스를 받아 아깝지 않게 건강 체크 할 수 있었어요~내원 전부터 이것저

In [36]:
driver.quit()